# Project: Sentiment Analysis Sorting: Summerizing and Sorting Yelp Reviews

## Group: Wisefish 

* Wenhao Zhang, wenhaoz 
* Graeme Milne, gmilne 
* Mitchell McCormack, mmccorma
* Jonathan Lo, jcl60

### SemSort Overview

SemSort utilizes two natural language processing artificial intelligence techniques to automatically aggregate reviews for given business into quickly digestible lists. Review summarization is done using an improved SumBasic algorithm, a multidocument summarization tool. Semantic Analysis is carried out by a neural network trained to predict the positive or negative sentiment of sentence. SemSort is a tool to provide additional subjective information to reviews, offering an extension to Yelp’s star rating system.

### Motivation 

The motivation for SemSort came out of an interest to learn more about the topics of sentiment analysis and text summarization. Our group wanted to test the effectiveness of different models all the while implementing something practical. After viewing the Yelp dataset we came up with the idea of SemSort. Once we confirmed that this was a relatively unique combination of the two NLP areas of study, we decided to to pursue it as our project.  

### Data 

Data files used in this project include: 
* [The Yelp Dataset](https://www.yelp.com/dataset)
* [The Opinosis Dataset](https://github.com/kavgan/opinosis/blob/master/OpinosisDataset1.0_0.zip)

### Code 

Code used in this project that was not our own includes: 

* [rougescore.py](https://github.com/bdusell/rougescore/blob/master/rougescore/rougescore.py)
* [LSTM RNN](https://www.kaggle.com/ngyptr/lstm-sentiment-analysis-keras)

### Development Process



In order to come up with an idea, the group got together a couple times to brainstorm. We used the provided topics and links as a starting point. After discussing several topics and potential datasets, we eventually settled on using text summarization and sentiment analysis on the yelp reviews dataset. We drew the required pipeline that we would need to follow (see SemSort Pipeline below). After that, we divided up the tasks and set a deadline for ourselves and another meeting date to discuss any final obstacles as well as notebook and poster documentation. 

### SemSort Pipeline

![title](SemsortpipelineColor.png)

## SemSort

### Semantic Sorting

The Sorting function takes in an array of sentences vectorizes them then uses the `model.predict()` function to get a 2 element vector returned for each sentence. The first element in the vector is a float showing the predicted negative sentiment and the second element is the predicted positive sentiment. The sort algorithm then takes the higher of the two values and places the sentence in output dictionary accordingly.

In [ ]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import load_model
import numpy as np
import pandas as pd
import progressbar

def basicSentimentSort(sentences, with_sentiment=False):
    #setup
    model = load_model('model5.h5')
    # most common words used as features
    max_features = 3000
    # Tokenizer for preprocessing sentence to wordVecs 
    tokenizer = Tokenizer(num_words=max_features, split=' ')
    print("Sorting: ")
    bar = progressbar.ProgressBar(max_value=len(sentences))

    sorted_sentences = {"Positive":[], "Negative":[]}

    for i, sentence in enumerate(sentences): 
        # vectorize the sentence
        tokenized_sentence = tokenizer.texts_to_sequences(sentence)
        # pad the sentences for the network 
        tokenized_sentence = pad_sequences(tokenized_sentence, maxlen=944, dtype='int32', value=0)
        # generate the sentiment prediction from the model -> 2 element list [negative <float> , positive <float>]
        sentiment = model.predict(tokenized_sentence,batch_size=1,verbose = 2)[0]
        #place sentence in the output dict
        if sentiment[1] > sentiment[0]:
            if with_sentiment:
                sorted_sentences["Positive"].append((sentence, sentiment))
            else:
                sorted_sentences["Positive"].append(sentence)  
        else:
            if with_sentiment:
                sorted_sentences["Negative"].append((sentence, sentiment))
            else:
                sorted_sentences["Negative"].append(sentence)  
        bar.update(i)
    return sorted_sentences

voteSentimentSort was an attemp to leverage three models. The first model was accurate at predicting the positive sentimemnt but not very accurate at predicting the negative sentiment. The Second model was the inverse of hte first model and the third model was fairly accurate for both positive and negative but not as accurate as teh first two mdoels in their respective categories. 

A sentence is passed the the `model.predict()` for each model then the argmax of the average of hte predicted sentiment is used to place the model in the output dictionary. 

In [ ]:
def voteSentimentSort(sentences):
    # good at finding positive sentiment
    model1 = load_model('D:\\SFU\\cmpt-413\\nlpclass-1187-g-wisefish\\project\\yelpModel\\model1.h5')
    # good at finding negative sentiment
    model2 = load_model('D:\\SFU\\cmpt-413\\nlpclass-1187-g-wisefish\\project\\yelpModel\\model2.h5')
    model3 = load_model('D:\\SFU\\cmpt-413\\nlpclass-1187-g-wisefish\\project\\yelpModel\\model5.h5')
    max_features = 3000
    tokenizer = Tokenizer(num_words=max_features, split=' ')

    sentence_sentiment = {}
    print("\nPredict with Model 1: ")
    bar1 = progressbar.ProgressBar(max_value=len(sentences))
    for i, sentence in enumerate(sentences): 
        #vectorizing 
        tokenized_sentence = tokenizer.texts_to_sequences(sentence)
        #padding 
        tokenized_sentence = pad_sequences(tokenized_sentence, maxlen=975, dtype='int32', value=0)
        sentiment = model1.predict(tokenized_sentence,batch_size=1,verbose = 2)[0]
        sentence_sentiment[sentence] = [sentiment[0], sentiment[1]]
        bar1.update(i)
    print("\nPredict With Model 2: ")
    bar2 = progressbar.ProgressBar(max_value=len(sentences))
    for i, sentence in enumerate(sentences):
        #vectorizing 
        tokenized_sentence = tokenizer.texts_to_sequences(sentence)
        #padding 
        tokenized_sentence = pad_sequences(tokenized_sentence, maxlen=273, dtype='int32', value=0)
        sentiment = model2.predict(tokenized_sentence,batch_size=1,verbose = 2)[0]
        sentence_sentiment[sentence].append(sentiment[0])
        sentence_sentiment[sentence].append(sentiment[1])
        bar2.update(i)
    
    print("\nPredict With Model 3: ")
    bar3 = progressbar.ProgressBar(max_value=len(sentences))
    for i, sentence in enumerate(sentences):
        #vectorizing 
        tokenized_sentence = tokenizer.texts_to_sequences(sentence)
        #padding 
        tokenized_sentence = pad_sequences(tokenized_sentence, maxlen=946, dtype='int32', value=0)
        sentiment = model3.predict(tokenized_sentence,batch_size=1,verbose = 2)[0]
        sentence_sentiment[sentence].append(sentiment[0])
        sentence_sentiment[sentence].append(sentiment[1])
        bar3.update(i)
 
    print("\nSorting: ")
    sorted_sentences = {"Positive":[], "Negative":[]}
    for sentence, sentiment in sentence_sentiment.items():
        print(sentiment)
        m1_neg = sentiment[0]
        m1_pos = sentiment[1]
        m2_neg = sentiment[2]
        m2_pos = sentiment[3]
        m3_neg = sentiment[4]
        m3_pos = sentiment[5]

        avg_pos =  (m1_pos + m2_pos + m3_pos) / 3
        avg_neg =  (m1_neg + m2_neg + m3_neg) / 3 
        
        if avg_neg < avg_pos: 
            sorted_sentences["Positive"].append(sentence)
        else:
            sorted_sentences["Negative"].append(sentence)
    return sorted_sentences

### Full Pipeline Combination

Two SemSorts were implemented. The first is a single path algorithm that will summarize reviews then sort the summarized sentences into positive and negative categories and return the sorted summarized sentences. The multipath algorithm will split up the reviews based upon the start rating then run the single path on the lowest rated reviews and the highest rated reviews before merging them and taking the top *n* sentences to return. 

The multipath algorithm was implemented to account for the case where positive sentences existed in low star reviews or the inverse of negative sentences existing in high star reviews.


In [ ]:
from sum_basic import SumBasic
from yelpModel.sentimentSort import basicSentimentSort,voteSentimentSort
import pandas as pd

def pickTop(n, reviews, idx):
    sorted_reviews = sorted(reviews,  key=lambda tup: tup[1][idx],reverse=True)
    top = []
    for i in range(n):
        if i < len(sorted_reviews):            
            top.append(sorted_reviews[i][0])
    return top

def SemSort(data, multipath=False, num_bullet_points=4):
    sorted = {"Positive":[], "Negative":[]}
    if multipath:
        
        #Split Reviews by star rating
        oneStar = []
        twoStar = []
        fourStar = []
        fiveStar = []
        for _, row in data.iterrows():
            if row["Stars"] == 1:
                oneStar.append(row["Text"])
            elif row["Stars"] == 2:
                twoStar.append(row["Text"])
            elif row["Stars"] == 4:
                fourStar.append(row["Text"])
            elif row["Stars"] == 5:
                fiveStar.append(row["Text"])
        
        low_reviews = oneStar + twoStar
        high_reviews = fiveStar + fourStar
        
        #Summarize low start reviews
        neg_summarizer = SumBasic(low_reviews)
        neg_summary = neg_summarizer.get_summary(10)
        # Sort low star reviews
        neg_sorted = basicSentimentSort(neg_summary, with_sentiment=True)
        #Summarize high star reviews
        pos_summarizer = SumBasic(high_reviews)
        pos_summary = pos_summarizer.get_summary(10)
        #Sort high star reviews
        pos_sorted = basicSentimentSort(pos_summary, with_sentiment=True)

        total_pos = pos_sorted["Positive"] + neg_sorted["Positive"]
        total_neg = pos_sorted["Negative"] + neg_sorted["Negative"]
        
        #Merge the reviews by taking the top n from the positive and negative sentences
        sorted["Positive"] = pickTop(num_bullet_points, total_pos, 1)
        sorted["Negative"] = pickTop(num_bullet_points, total_neg, 0)
            

    else:
        # collect reviews into list of strings
        data_to_be_sorted = []
        for _, row in data.iterrows():
            data_to_be_sorted.append(row["Text"])

        #generate summaries of the reviews
        summarizer = SumBasic(data_to_be_sorted)
        summary = summarizer.get_summary(num_bullet_points)
        #sort summaries with RNN preditctions
        sorted = basicSentimentSort(summary)

    return sorted 
csv_data = pd.read_csv('CF_Toronto_Eaton_Centre_reviews.csv')

sorted = SemSort(csv_data, multipath=False, num_bullet_points=10)

print("\nPositive")
for i, sentence in enumerate(sorted["Positive"]):
    print(f'{i+1}: {sentence}')

print("\nNegative")
for i, sentence in enumerate(sorted["Negative"]):
     print(f'{i+1}: {sentence}')


### Preprocessing 

We are using the Yelp Academic DataSet (https://www.yelp.com/dataset/challenge) as our primary data source for reviews.
This dataset contains around 188,000 businesses and 6,000,000 reviews that were featured in Yelp at the time of collection.

For our evaluation purposes, we are only wanting to look at a subset of businesses and a subset of review data, namely the number of stars and the review's text content:
1. Load in the businesses listings and filter to only those in Toronto, Ontario, Canada with over 100 reviews.
2. Load in the review data, filter to only those matching the above businesses, and keep only the star rating and text content.
3. Retrieve all the review text contents for each business and feed it into the SumBasic and SemSort components.

### Preprocessing input for summarization

The class used for preprocessing the input for summarization is called ProcessData. The initialization function takes in 6 inputs. The first input variable data, is the data that needs to be processed. The type_pos_tag variable determines the type of POS tagger to be used. The default input is NLTK, this used the built in pos tagger from nltk. Setting this variable to Stanford will switch to using the Stanford pos tagger. The remove_short feature removes sentences that are too short. The lemmatize variable enables or disables lemmatization. True will use lemmatization. Lemmatization is the process of grouping together the inflected forms of a word so they can be analyzed together as a single unit. We found this to help the quality of summarization from SumBasic. The remove_stop_word variable will cause stop words to be removed if it is set to true. Stop words are those words that do not provide much information. In our case we justed the NLTK english stop words. The remove_punc variable will remove all punctuations if it is set to true. This was helpful for SumBasic as we found if we did not remove the punctuation, they would skew the results. Some of the reviews on Yelp were just some punctuations.

The first part of the initialization is to remove the short sentences. This removes any sentences that are less than length of 2. In the next step we tokenize the sentences.

In [ ]:
   def __init__(self, data, type_pos_tag = "NLTK", remove_short = True, lemmatize = True, remove_stop_word = True, remove_punc = True):

        self.puncs = [".",",",";",":","!","?","``","''","'","-"]
        self.lemmatizer = WordNetLemmatizer()
        self.stopwords = stopwords.words("english")
        self.sentences = []
        self.type_pos_tag = type_pos_tag
        self.pos_tagger = StanfordPOSTagger('./stanford/models/english-bidirectional-distsim.tagger','./stanford/stanford-postagger.jar')
        self.lemmatize = lemmatize
        self.remove_stop_word = remove_stop_word
        self.remove_punc = remove_punc

        if remove_short:
            for i, line in enumerate(data):
                if len(line) < 2:
                    del data[i]

        for line in data:
            tokens = sent_tokenize(line)
            for s in tokens:
                if s not in self.sentences:
                    self.sentences.append(s)

The following function is used to remove any punctuations. It removes any punctuation in the following list: [".",",",";",":","!","?","``","''","'","-"].

In [ ]:
    def _remove_punc(self, sent):
        
        for punc in self.puncs:
            sent = list(filter(lambda a: a[0] != punc, sent))
       
        return sent

This function converts a treebank tag to wordnet tag. This is needed as the lemmatizer from NLTK takes in a wordnet tag.

In [ ]:
   def _get_wordnet_pos(self, treebank_tag):
        switch = {"J": wordnet.ADJ, "V": wordnet.VERB, "R": wordnet.ADV}
        return switch.get(treebank_tag, wordnet.NOUN)

This function cleans the inputs for summarization. The function first tokenizes the words in the sentences. It then does pos tagging based on what pos tagger is being used. It then lower cases the words so they will be treated the same. Then the puncuations are removed. Next the top words are remove and lemmatization is performed if it is needed. 

In [ ]:
def clean_sentences(self):
        cleaned = []
        
        for sentence in self.sentences:
            words = word_tokenize(sentence)

            #words = [w.lower() for w in words]

            #pos tagging
            if self.type_pos_tag == "Stanford":
                tokens = []
                for w in words:
                    tokens.extend(self.pos_tagger.tag([w]))

            else:
                tokens = []
                for w in words:
                    tokens.extend(pos_tag([w]))

            tokens = [(t[0].lower(),t[1]) for t in tokens]

            #remove punctuations
            if self.remove_punc:
                tokens = self._remove_punc(tokens)

            #lemmatize
            if self.lemmatize:
                tokens = [(self.lemmatizer.lemmatize(t[0], self._get_wordnet_pos(t[1])),t[1]) for t in tokens]

            #remove stop words
            if self.remove_stop_word:
                tokens = [t for t in tokens if t[0] not in self.stopwords]

            cleaned.append(tokens)
        
        return cleaned

This function removes the pos tags from the data set as the clean_sentences function returns a list of tuples containing the word and its pos tag.

In [ ]:
    def remove_tags(self, data):
        cleaned = []

        for s in data:
            temp = [w[0] for w in s]
            cleaned.append(temp)
        
        return cleaned

### SumBasic 


SumBasic is a summerization system for multi document input that uses word probability to determine the importance of sentences. Each sentence is assigned a weight equal to the average probability of the words in it. Then the best scoring sentence is selected. The algorithm runs until the generated summary meets the expected length. The full steps to the SumBasic algorithm are shown below.

$$
\textbf{Step 1}\\
\text{Compute the probability distribution over the words \(w_i\) appearing in the input, \(p(w_i)\) for every \(i\); \(p(w_i)\) = \(\frac{n}{N}\), where \(n\) is the number of times the word appeared in the input, and \(n\) is the total number of content word tokens in the input.}\\
$$


$$
\textbf{Step 2}\\ 
\text{For each sentence SJ in the input, assign a weight equal to the average probability of the words in the sentence, i.e.}\\
$$

$$
weight (S_j) = \sum_{w_i) \in{S_j}} \frac{p(w_i}{|\{w_i|w_i\in{S_j}\}|} \\ 
$$

$$\textbf{Step 3}\\ \text{Pick the best scoring sentence that contains the highest probability word.}\\ 
$$

$$ \textbf{Step 4}\\ \text{For each word \(w_i\) in the sentence chosen at step 3, update their probability} 
$$ 

$$ p_{new}(w_i) = p_{old}(w_i) * p_{old}(w_i)\\ $$
$$ \textbf{Step 5}\\ \text{If the desired summary length has not been reached, go back to Step 2.}
$$

The class for performing SumBasic summarization is called SumBasic. This class takes in one input, the data to be summarized. In the init function the class creates a copy of the data to be later used to output the summaries. The sentences are then sent to the data processing class to be prepared for use in the summarization.

In [ ]:
    def __init__(self, data):
        self.data = []

        for sentence in data:
            tokens = sent_tokenize(sentence)
            for s in tokens:
                if s not in self.data:
                    self.data.append(tokens)
        
        self.sentence_weights = {}

        data_processor = ProcessData(data)
        
        self.sentences = data_processor.remove_tags(data_processor.clean_sentences())

        self.probabilities = self._get_probabilities(self.sentences)

This function updates the probability of each word in the sentence that won. The score updated with the old probability of the word squared.

In [ ]:
    def _update_probabilities(self, winner):
        for word in winner:
            self.probabilities[word.lower()] = self.probabilities[word.lower()] * self.probabilities[word.lower()]

The get probability function intiailizes the probabilities for the SumBasic summarization. The program counts the number of times that each words appears in all the sentences. Then it divides the count of each individual word by the total number of tokens in the entire data set.

In [ ]:
def _get_probabilities(self, data):
        
        word_probabilities = {}
        token_count = 0

        for sentence in data:

            for word in sentence:
                token_count += 1

                if word in word_probabilities:
                    word_probabilities[word.lower()] += 1
                else:
                    word_probabilities[word.lower()] = 1

        for word in word_probabilities:
            word_probabilities[word] = word_probabilities[word] / token_count

        return word_probabilities

The _weight_sentence function get the weight of the sentence being looked at. The sentence weight is defined as the sum of the weight of all the probabilites of each word in the sentence divided by the total amount of words in the sentence. If the token_count is zero, the function will return 0 for the weight

In [ ]:
 def _weight_sentence(self, sentence):
        sentence_count = 0
        token_count = 0
        for word in sentence:
            if word in self.probabilities:
                sentence_count += self.probabilities[word]
                token_count += 1
        
        return sentence_count/token_count if token_count > 0 else 0

The following function will look throught all the sentences and find those that contain the highest probability word and return the most likely sentence. It first finds the most likely word. It then finds all the sentences that contain the most likely word. It then looks through the candidates and determines the sentence with the highest sentence weight.

In [ ]:
def _get_max_sentence(self):
        highest_prob_word = max(self.probabilities.items(), key=operator.itemgetter(1))[0]

        sentences_containing_highest_prob_word = []
        for sentence in self.sentences:
            if highest_prob_word in sentence:
                sentences_containing_highest_prob_word.append(sentence)

        winner = ""
        winner_prob = 0

        for sentence in sentences_containing_highest_prob_word:
            if self.sentence_weights[tuple(sentence)] > winner_prob:
                winner_prob = self.sentence_weights[tuple(sentence)]
                winner = sentence

The following function is used to get the summary. The function loops through the data until a desired about of sentences for the summary is reached or there is no more data to be processed. For each of the sentences, it gets the weight of the sentence for the current iteration. It then gets the sentence with max weight. Then it adds the sentence to the summary and repeats until the desire length is reached.

In [ ]:

    def get_summary(self, length):
        
        summary = []

        while len(summary) < length and len(self.data) > 0:
            for sentence in self.sentences:
                self.sentence_weights[tuple(sentence)] = self._weight_sentence(sentence)

            winner = self._get_max_sentence()
            winner_index  = self.sentences.index(winner)

            summary.append(self.data[winner_index][0])

            if winner != '':
                self._update_probabilities(winner)
                del self.sentences[winner_index]
                del self.data[winner_index]

        return summary

### Opinosis Graph

The OpinosisGraph class generates the graph that the OpinosisSummarizer uses. It takes in the data and two other inputs. These tell the class to use lemmatization or to remove the stop words. The first thing the init function does is call the ProcessData class to get the data preprocessed for summarizatoin.

In [ ]:
    def __init__(self, data, remove_stop_word = False, lemmatize = True):
        self.sentences = []
        self.graph = {}
        self.PRI = {}
        processor =  ProcessData(data,"Stanford", False, lemmatize, remove_stop_word, False)

        self.sentences = processor.clean_sentences()

The following function generates the graph. The function loops through each sentence and adds each word in the sentence to the graph. Each node of the graph contains a list of PRI, Positional Reference Information, and the word with its pos tag. The SID of a word is the sentence identifier, or the sentence it appears in. PID is the position of occurence of the location in the sentence it appeared in. Together this forms a PRI. If the node exists, the list of PRI is just updated with the new location. If it does not exist a new node is created. 

In [ ]:
    def generate_opinosis_graph(self):
        n = len(self.sentences)
        for i in range(n):
            words =  self.sentences[i]
            
            for (j,(word,pos)) in enumerate(words):
                if word == "'s" and pos == "VBZ":
                    words[j] = ("is", pos)
                if word == "n't" and pos == "RB":
                    words[j] = ("not", pos)
                if word == "wa" and pos == "VBD":
                    words[j] = ("was", pos)
                if word == "ca" and pos == "MD":
                    words[j] = ("can", pos)

            sent_size = len(words)
            for j in range(sent_size):
                LABEL = words[j]
                PID = j
                SID = i
                if LABEL in self.PRI:
                    self.PRI[LABEL].append((SID, PID))
                else:
                    self.graph[LABEL] = []
                    self.PRI[LABEL] = [(SID, PID)]
                
                prev_node = words[j-1]

                if not self._edge_exists(prev_node, LABEL) and j > 0:
                    self.graph[prev_node].append(LABEL)

This function checks if the edge between the previous node and current node already exists. If it exists a new node will not be created. Since the graph is based on a dictionary, the function gets the list of nodes from the prev_node and check if the current is in the list. If it is, it returns True otherwise False.

In [ ]:
    def _edge_exists(self, prev_node, curr_node):
        if prev_node in self.graph:
            edges = self.graph[prev_node]
            if curr_node in edges:
                return True

        return False

### Opinosis

Opinosis is an abstractive summarization method that makes use of graphs to generate summaries of highly redundant opinions. Unlike many other methods Opinosis does not require any domain knowledge and only uses shallow NLP processing.

The class for Opinosis summarization is called OpinosisSummarizer. The first thing the summarizer gets its the graph for the summary from the OpinosisGraphc class.

In [ ]:
def __init__(self, data, sigma_ss, sigma_r, sigma_gap, sigma_vsn, collapse, similarity, remove_stop_word=False, lemmatize=True):
        self.sigma_ss = sigma_ss
        self.sigma_r = sigma_r
        self.sigma_gap = sigma_gap
        self.sigma_vsn = sigma_vsn
        self.collapse = collapse
        self.similarity = similarity

        graph = OpinosisGraph(data, remove_stop_word, lemmatize)
        graph.generate_opinosis_graph()

        self.graph = graph.graph
        self.PRI = graph.PRI

This function determines if a node is a valid start node. A node is a valid start location of the average of the PID of the node are less than or equal a parameter that is an user input, sigma_vsn. If it is less than or equal to sigma_vsn, the node must also be a natural starting point. This include words like its, the, when, a, an, this, the, they, it ,i, we, and our. A valid start pos tag is JJ, RB, PRP$, VBG, NN, and DT. If all of the conditions are met, the node is a valid start.

In [ ]:
 def _is_valid_start_node(self, node):
        PRI_node = self.PRI[node]

        common_start_words = "r^(its/|the/|when/|a/|an/|this/|the/|they/|it/|i/|we/|our/).*"

        valid_start_tags = ["/JJ", "/RB", "/PRP$", "/VBG", "/NN", "/DT"]

        len_PRI = len(PRI_node)

        total = 0

        for PRI in PRI_node:
            total += PRI[1]

        word_tag = node[0]+"/"+node[1]

        if total/len_PRI <= self.sigma_vsn:
            if re.match(common_start_words, word_tag, re.I) or "it/PRP" in word_tag or "if/" in word_tag or "for/" in word_tag:
                return True
            for v_t in valid_start_tags:
                if v_t in word_tag:
                    return True

This function determines if the node is a valid end node. It is a valid end if the node ahs a pos tag of ., ,, or CC.

In [ ]:

    def _is_valid_end_node(self, node):
        if node[1] == "." or node[1] == "," or node[1] == "CC":
            return True

        return False


The folloing function determines if the path is a valid path. It is a valid path if it satisfies one of the regex.

In [ ]:
    def _is_valid_path(self, sentence):

        well_formed = False

        pos_sent = ""

        for w in sentence:
            pos_sent += "/" + w[1]

        if re.match(".*(/JJ)*.*(/NN)+.*(/VB)+.*(/JJ)+.*", pos_sent, re.I):
            well_formed = True
        elif not re.match(".*(/DT).*", pos_sent, re.I) and re.match(".*(/RB)*.*(/JJ)+.*(/NN)+.*", pos_sent, re.I):
            well_formed = True
        elif re.match(".*(/PRP|/DT)+.*(/VB)+.*(/RB|/JJ)+.*(/NN)+.*", pos_sent, re.I):
            well_formed = True
        elif re.match(".*(/JJ)+.*(/TO)+.*(/TO)+.*(/VB).*", pos_sent, re.I):
            well_formed = True
        elif re.match(".*(/RB)+.*(/IN)+.*(/NN)+.*", pos_sent, re.I):
            well_formed = True

        last = sentence[-1][1]

        #if re.match("(/TO|/VBZ|/IN|/CC|/PRP|/DT|/,)", last, re.I):
        #    well_formed = False

        return well_formed

This function does scoring for each incremental step.

In [ ]:
   def _Swt_loglen(self, redudancy, L):
        return math.log(L, 2)*redudancy if L > 1 else redudancy

This function checks if a node is a hub node that can be a candidate for collapsing. A node is collapsible if it is a VB, this is because verbs are common in reviews.

In [ ]:
    def _is_collapsible(self, node):
        return bool(re.match(r"VB", node[1], re.I))

This function find the intersection of sids such that the pids are less than a value sigma_gap. This determines if a path is redundant. The sigma_gap value determines the maximum gap that can be allowed to finding redundant paths.

In [ ]:
   def _intersect(self, PRI_n, PRI_overlap):
        PRI_intersect = []
        for sid_o, pid_o in PRI_overlap:
            for sid, pid in PRI_n:
                if sid_o == sid:
                    if pid > pid_o and abs(pid - pid_o) <= self.sigma_gap:
                        PRI_intersect.extend([(sid, pid)])
                        break
                else:
                    if sid > sid_o:
                        break

        return PRI_intersect

This function determines the Jaccard similarity of the two sentences. 

In [ ]:
    def _jaccard_similarity(self, candidate1, candidate2):

        if set(candidate1).issubset(set(candidate2)) or set(candidate2).issubset(set(candidate1)):
            return 1

        intersection_size = len(set(candidate1).intersection(set(candidate2)))
        union_size = len(set(candidate1).union(set(candidate2)))
        return intersection_size/union_size

This function is used to remove the duplicates of the list of candidates. These duplicates are near duplicates. This means that they are very close to each other. The algorithm used in this duplicate removal function is similar to an agglomerative clustering algorithm. First the function calculates the Jaccard index between half the pairs of the sentences. The Jaccard index is used to determine if the sentences are near duplicates. Next the pair with the highest Jaccard index is found. If the value is greater than similarity, they are clustered together. The algorithm then repeats the process until there is no change in the number of clusters. After the function finds the sentence with the highest score for each cluster. These sentences are returned as the final candidates without duplicates.

In [ ]:
   def _remove_duplicates(self, candidates):

        clusters = [[key] for key, value in candidates.items()]
        final_sentences = []

        prev_size = len(clusters)
        curr_size = prev_size * 2
        while curr_size != prev_size and len(clusters) > 1:

            prev_size = len(clusters)

            scores = []
            temp = []

            size_cluster = len(clusters)

            for i in range(size_cluster):

                c1 = clusters[i]

                for j in range(i + 1, size_cluster):

                    c2 = clusters[j]
                    best_score = self._jaccard_similarity(c1[0], c2[0])

                    for a in c1:
                        for b in c2:
                            score = self._jaccard_similarity(a, b)
                            if score > best_score:
                                best_score = score

                    key = c1+c2
                    temp.append([c1, c2])
                    scores.append((key, best_score))

            highest = max(scores, key=lambda x: x[1])
            highest_index = scores.index(highest)

            if highest[1] > self.similarity:
                clusters.append(highest[0])

                for i in temp[highest_index]:
                    clusters.remove(i)

            curr_size = len(clusters)

        for cluster in clusters:
            max_sentence = max(cluster, key=candidates.get)
            final_sentences.append(tuple(max_sentence))

        return final_sentences

This function is used to calculate the path score for a group of collapsed candidates. It is the average of all the scores of the ccs.

In [ ]:
    def _average_path_score(self, cc, score):

        len_cc = len(cc)

        scores = 0

        for i in cc:
            scores += score[i]

        return scores/len_cc

This function is used to stitch together all the collapsed candidates. It first appends all the cc except the last one with commas. For the laste one, a coordinating conjuction is needed. It looks through all the coordinating conjuctions and determines the one with the highest redundancy. The winner is used to connect the last cc.

In [ ]:
   def _stitch(self, anchor, cc):
        new_sent = anchor[:]

        if len(cc) > 1:
            for s in cc[:-1]:
                for w in s:
                    new_sent += (w,)
                new_sent += ((",", ","),)
            last_node = cc[-1][0]

            cc_nodes = [("and", "CC"), ("for", "IN"), ('nor', 'CC'),
                        ('but', 'CC'), ('or', 'CC'), ('yet', 'RB'), ('so', 'RB')]

            highest_r = 0
            best_cc = cc_nodes[0]

            for node in cc_nodes:
                try:
                    r = len(self._intersect(
                        self.PRI[node], self.PRI[last_node]))
                    if r > highest_r:
                        highest_r = r
                        best_cc = node
                except(KeyError):
                    pass

            new_sent += (best_cc,)

            for w in cc[-1]:
                new_sent += (w,)

        else:
            for w in cc[0]:
                new_sent += (w,)

        return new_sent

This function is used to traverse the nodes to find candidate sentences. If the node is not redundant enough nothing is done. If it is, the function first checks if the the sentence is a valid sentence. If it is, it is added along with its score and the function returns the list of candidates. If not the function will loop through each of the neighbours of the current node. If the neighbour is a candidate for collapse, it will collapse the node, otherwise it will call _traverse to continue looking for candidate sentences.

In [ ]:
    def _traverse(self, candidates, node, score, PRI_overlap, sentence, length):
        redundancy = len(PRI_overlap)
        if redundancy >= self.sigma_r:
            if self._is_valid_end_node(node):
                if self._is_valid_path(sentence):
                    final_score = score/length
                    candidates[sentence] = final_score

            for v_n in self.graph[node]:
                PRI_new = self._intersect(self.PRI[v_n], PRI_overlap)
                r = len(PRI_new)
                new_sentence = sentence + (v_n,)
                L = length + 1
                new_score = score + self._Swt_loglen(r, L)
                if self._is_collapsible(v_n) and self.collapse:
                    c_anchor = new_sentence
                    tmp = {}
                    for v_x in self.graph[v_n]:
                        self._traverse(tmp, v_x, 0, PRI_new, (v_x,), L)

                        if tmp:
                            cc = self._remove_duplicates(tmp)
                            cc_path_score = self._average_path_score(cc, tmp)
                            final_score = new_score + cc_path_score
                            sitch_sent = self._stitch(c_anchor, cc)
                            print(sitch_sent)
                            candidates[sitch_sent] = final_score
                else:
                    self._traverse(candidates, v_n, new_score,
                                   PRI_new, new_sentence, L)

This function is for getting the summary from Opinosis. The function will loop through each node in the graph. It determines if the node is a valid start node, if it is it will call _traverse to try to produce a candidate sentence. Once the _traverse returns the candidates are added to the list of candidates. Once the loop is complete the function removes the duplicate candidates and gets the top sentences. The number returned is determines by sigma_ss. The candidates is turned into sentences and returned.

In [ ]:
  def get_summary(self):
        candidates = {}
        summaries = []

        for v_j in self.graph.keys():

            if self._is_valid_start_node(v_j):

                path_length = 1
                score = 0
                clist = {}
                self._traverse(clist, v_j, score,
                               self.PRI[v_j], (v_j,), path_length)
                candidates.update(clist)

        final = self._remove_duplicates(candidates)
        top = sorted(final, key = candidates.get, reverse=True)[:self.sigma_ss]
        #top = sorted(final, key=candidates.get, reverse=True)

        for s in top:
            summary = ""
            length = len(s)
            for i in range(length-1):
                summary += s[i][0]

                if s[i+1][1] == ".":
                    summary += s[i+1][0]
                elif (s[i+1][0] == "," or s[i+1][0] == "and") and i + 1 == length:
                    summary += "."
                elif s[i+1][1] != ",":
                    summary += " "

            summaries.append(summary)

        return summaries

### Experimental Setup #1 

### Validation of SumBasic and Opinosis Implementation

In order to test that our SumBasic and Opinosis implementation are outputting useful data we use the ROUGE evaluation metrics. ROUGE stands for recall-orientated understudy for gisting evaluation and is often used for evaluating machine text summarization and machine translation ( https://en.wikipedia.org/wiki/ROUGE_(metric) ). We specifically look at Rouge-N which measures the overlap in the n-grams between our generated summary and one or more gold standard models in the dataset. The dataset that we used for testing is the [Opinosis dataset](http://kavita-ganesan.com/opinosis-opinion-dataset/#.XAM97BB6ov4) which has an attached research paper with which we can compare results.

### ROGUE in practice

In the ROGUE evaluation we have two items: 
- A machine generated summary (output of SumBasic) 
- A set of one or more gold standard reference summaries

Once we have these we can caluclate the ROUGE-1 and ROUGE-2 precision and recall. The following is a toy example adapted from a blog post found on [www.RXNLP.com](https://rxnlp.com/how-rouge-works-for-evaluation-of-summarization-tasks/#.XANBwhB6ov4)

Example machine generated summary = "the dog ate all my homework" 
Example gold standard reference model = "the dog ate my homework" 

$$\text{ROUGE-N Recall} = \dfrac{\text{number of overlapping n-grams}}{\text{number of words in gold standard}}$$

$$\text{ROUGE-N Precision} = \dfrac{\text{number of overlapping n-grams}}{\text{number of words in generated summary}}$$

Using our toy example to calculate ROGUE-1(unigram) precision and recall outputs the following results:

$$\text{ROUGE-1 Recall} = \dfrac{\text{5}}{\text{5}} = 1.0 $$

$$\text{ROUGE-1 Precision} = \dfrac{\text{5}}{\text{6}} = 0.8333... $$

These results can also be obtained by using [rougescore.py](https://github.com/bdusell/rougescore/blob/master/rougescore/rougescore.py). Note that, the 3rd parameter, referred to as α is a number between 0 and 1 where 0 favors recall and 1 favors precision.  

In [6]:
import rougescore

summary_test = ["the", "dog", "ate", "all", "my", "homework"]
reference_test = [["the", "dog", "ate", "my", "homework"]]
rouge1_recall_test = rougescore.rouge_1(summary_test, reference_test, 0)
rouge1_precision_test = rougescore.rouge_1(summary_test, reference_test, 1)

print("Rouge-1 Recall: = " + str(rouge1_recall_test))
print("Rouge-1 Precision:  = " + str(rouge1_precision_test))

Rouge-1 Recall: = 1.0
Rouge-1 Precision:  = 0.8333333333333334


### The Results #1

### Precision: 

$$
\begin{array}{rr} \hline
    \hline
    & ROGUE-1 & ROGUE-2 & Avg Word Count \\[0.5ex]
    \hline
    SumBasic & 0.03037 & 0.0777 &  34 \\
    \hline
    Opinosis Implementation & ??? & ??? & ??? \\
    \hline
    Opinosis Paper & 0.2831 & 0.0853 & 15 \\
    \hline
\end{array}
$$


### Recall: 
$$
\begin{array}{rr} \hline
    \hline
    & ROGUE-1 & ROGUE-2 & Avg Word Count \\[0.5ex]
    \hline
    SumBasic & 0.1538 & 0.0377 & 34 \\
    \hline
    Opinosis Implementation & ??? & ??? & ??? \\
    \hline
    Research Results & 0.4482 & 0.1416 & 15 \\
    \hline
\end{array}
$$

### Sentiment Analysis 


Semantic sorting is carried out by passing each sentence returned from the extended SumBasic algorithm into a model to predict if the sentence is positive or negative. The model was created using a Long Short Term Memory Recursive Neural Network. The training data was preprocessed by sorting a sample of yelp reviews into positive and negative categories based the star rating the review had. One and Two stars were considered a negative review. Four and five stars were considered a positive review. Then Three methods were tested for training the model.




### Experimental Setup #2 

The first method was to pass the entire text of the review through the network with it's label of 'Positive' or 'Negative'. The second method was to split the text of the review into sentences and pass the sentences through the network individually with the corresponding label. The third method experimented with, passing all of the bigrams generated from each sentence as a bag of words. Method 3 was undertaken as an adaptation of FastText to a RNN and as the results show was the least effective.  

The sorting method is to take the sentences produced by the summarizing algorithm then pass each of them into a model to produce two values. The sentence is sorted by the argmax of the values produced by the prediction. Model 1 is used to produce sentiment predictions. Table 3 shows the considerable increase in accuracy Model 1 has over Models 2 and 3. 

### Formating Input Data

Split the input reviews by either 1 and 2 stars or 3 and 4 stars add the review text to with either a 'Positive' or 'Negative' label into a pandas dataframe. 

In [1]:
import pandas as pd 
from nltk.tokenize import sent_tokenize
import random 
import csv

def basicDataFormatting():
    dataframe = pd.read_csv('reviews.csv')
    sample_size = 10000
    neg_stars = [1,2]
    pos_stars = [4,5]

    pos = dataframe.loc[dataframe['stars'].isin(pos_stars)]
    neg = dataframe.loc[dataframe['stars'].isin(neg_stars)]

    pos_sample = pos.sample(sample_size)
    neg_sample = neg.sample(sample_size)

    print('Positive: ', pos_sample.size)
    print('Negative: ', neg_sample.size)

    sampled_data = []

    for i, row in pos_sample.iterrows():
        sampled_data.append(['Positive', row["text"]])
    for i, row in neg_sample.iterrows():
        sampled_data.append(['Positive', row["text"]])

    random.shuffle(sampled_data)
    outframe = pd.DataFrame(sampled_data, columns=['Sentiment', 'Text'])

    outframe.to_csv(f'sentiment_yelp_data_split_{sample_size*2}_sample.csv')
    print('Done')
    return

Use a nltk sentence tokenizer to split the reviews and repeat the process above to added sentence to a pandas dataframe with a 'Positive' or 'Negative' label.

In [ ]:
def sentenceSplitDataFormatting():
    dataframe = pd.read_csv('reviews.csv')
    sample_size = 5000
    neg_stars = [1,2]
    pos_stars = [4,5]

    pos = dataframe.loc[dataframe['stars'].isin(pos_stars)]
    neg = dataframe.loc[dataframe['stars'].isin(neg_stars)]

    pos_sample = pos.sample(sample_size)
    neg_sample = neg.sample(sample_size)

    print('Positive: ', pos_sample.size)
    print('Negative: ', neg_sample.size)

    sampled_data = []

    for _, row in pos_sample.iterrows():
        review = row["text"]
        tokenized_review = sent_tokenize(review)
        for sentence in tokenized_review:
            sampled_data.append(['Positive', sentence])
    for _, row in neg_sample.iterrows():
        review = row["text"]
        tokenized_review = sent_tokenize(review)
        for sentence in tokenized_review:
            sampled_data.append(['Negative', sentence])

    random.shuffle(sampled_data)
    outframe = pd.DataFrame(sampled_data, columns=['Sentiment', 'Text'])

    outframe.to_csv(f'sentiment_yelp_data_sentence_split_{sample_size*2}_sample.csv')
    print('Done')


    return

Sort the reviews based on star level, then split each review text by sentnece, then generate all of the bigrams present in the sentence and append the bigrams to the end of the sentence before adding to a pandas dataframe with 'Positive' or 'Negative' labels.

In [ ]:
def generate_ngrams(sentence, n):
    
    if n >= len(sentence):
        return sentence

    words = sentence.split(" ")
    ngrams = []
    
    for i in range(len(words)-n):
        ngram = ""
        for j in range(n):
            ngram = f'{ngram}{" " + words[i+j]}'    
        ngrams.append(ngram)
    words = words + ngrams
    return ' '.join(words)

def ngramDataFormatting():
    dataframe = pd.read_csv('reviews.csv')
    sample_size = 5000
    neg_stars = [1,2]
    pos_stars = [4,5]

    pos = dataframe.loc[dataframe['stars'].isin(pos_stars)]
    neg = dataframe.loc[dataframe['stars'].isin(neg_stars)]

    pos_sample = pos.sample(sample_size)
    neg_sample = neg.sample(sample_size)

    print('Positive: ', pos_sample.size)
    print('Negative: ', neg_sample.size)

    sampled_data = []

    for i, row in pos_sample.iterrows():
        review = row["text"]
        tokenized_review = sent_tokenize(review)
        for sentence in tokenized_review:
            ngram_sentence = generate_ngrams(sentence, 2)
            sampled_data.append(['Positive', ngram_sentence])
    for i, row in neg_sample.iterrows():
        review = row["text"]
        tokenized_review = sent_tokenize(review)
        for sentence in tokenized_review:
            ngram_sentence = generate_ngrams(sentence, 2)
            sampled_data.append(['Negative', ngram_sentence])

    random.shuffle(sampled_data)
    outframe = pd.DataFrame(sampled_data, columns=['Sentiment', 'Text'])

    outframe.to_csv(f'sentiment_yelp_data_ngram_split_{sample_size*2}_sample.csv')
    print('Done')

    return


### Training the Models

The RNN was created using keras and tensorflow libraries and the alogrithm was adapted from Peter Nagys write up on using a Long Short Term Memory RNN. The tuning variables adjusted were `max_features` and `batch_size` which were increased to 3000 and 64 respectively. The other two tuning features which were adjusted were `embed_dim` and `lstm_out` which were increased to 512 and 256. 

In [ ]:
# https://www.kaggle.com/ngyptr/lstm-sentiment-analysis-keras - algorithm

import numpy as np 
import pandas as pd 
from sklearn.feature_extraction.text import CountVectorizer
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense, Embedding, LSTM, SpatialDropout1D
from sklearn.model_selection import train_test_split
from keras.utils.np_utils import to_categorical
import csv

data = pd.read_csv('sentiment_yelp_data_sentence_split_20000_sample.csv')

max_features = 3000
print("Preprocessing Text Sequences ... ")
# Preprocess the text strings into vectors and pad the vectors to ensure consitent size 
tokenizer = Tokenizer(num_words=max_features, split=' ')
tokenizer.fit_on_texts(data['Text'].values)
text_sequences = tokenizer.texts_to_sequences(data['Text'].values)
text_sequences = pad_sequences(text_sequences)

print("Creating Model ...")
# generate the tensorflow RNN using a softmax funciton for the neruons 
model = Sequential()
model.add(Embedding(max_features, 512, input_length=text_sequences.shape[1]))
model.add(SpatialDropout1D(0.4))
model.add(LSTM(256, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(2,activation='softmax'))
model.compile(loss = 'categorical_crossentropy', optimizer='adam',metrics = ['accuracy'])
print(model.summary())

sentiments = pd.get_dummies(data['Sentiment']).values

# Partition the input data to training and testing
text_sequences_train, text_sequence_test, sentiments_train, sentiments_test = train_test_split(text_sequences,sentiments, test_size = 0.20, random_state = 42)
print(text_sequences_train.shape,sentiments_train.shape)
print(text_sequence_test.shape,sentiments_test.shape)

# Train the model for 5 epochs
batch_size = 64
model.fit(text_sequences_train, sentiments_train, epochs = 5, batch_size=batch_size, verbose = 1)

# Validate the model on the testing partition
validation_size = 1000

sentiments_validate = sentiments_test[-validation_size:]
text_sequence_validate = text_sequence_test[-validation_size:]
sentiments_test = sentiments_test[:-validation_size]
text_sequence_test = text_sequence_test[:-validation_size]

# Use model.predict() to output numpy vector of size 2, [negative <float>, positive <float>]
positive_count, positive_correct = 0, 0,
negative_count, negative_correct = 0, 0
for x in range(len(text_sequence_validate)):
    
    result = model.predict(text_sequence_validate[x].reshape(1,text_sequence_test.shape[1]),batch_size=1,verbose = 2)[0]
    
    if np.argmax(result) == np.argmax(sentiments_validate[x]):
        if np.argmax(sentiments_validate[x]) == 0:
            negative_correct += 1
        else:
            positive_correct += 1
       
    if np.argmax(sentiments_validate[x]) == 0:
        negative_count += 1
    else:
        positive_count += 1

# print results and save the model
print("Positive Accuracy: ", positive_correct/positive_count*100, "%")
print("Negative Accuracy: ", negative_correct/negative_count*100, "%")

model.save("model2.h5")
print('Done.')

### Results #2

Results of the internal validation testing during training on patritioned input data:



$$
\begin{array}{rr} \hline
    \hline
    Method & Positive & Negative \\[0.5ex]
    \hline
    Model 1 & 0.944 & 0.939 \\
    \hline
    Model 2 & 0.770 & 0.777 \\
    \hline
    Model 3 & 0.584 & 0.833 \\
    \hline
\end{array}
$$


### Analysis of the Results #2

Model 1 proved to be the most accurate model trained. The reason model 1 was the most successful was because the shape of the input data. Model took the whole review in and with the max features for the RNN set to 3000, the review provided more data points to assist in the prediction of positive and negative sentiment. 

Model 2 suffered from a lack of words in the sentence and therefore could not leverage the max features as much as Model 1. Model 3 was the least effective because adapting bigrams and the bag of words approach to a LSTM RNN resulted in a lot of repeated words per sentence. This added noise to the data being put through the network. The RNN by design considers the ngrams through the sentences history and by forcing the RNN to focus on bigrams by repeating them became problematic.

5 epochs were used for training because the training time per epoch varied between 45 minuets to 2 hours. 


### Final Ouput 

$$
\text{One example of output from the 252 reviews of Toronto's Eaton's Center}\\
$$
$$
\textbf{Positive:}
$$
* "The Eaton Centre does offer some positve exceptions, even considering the national demise of its namesake several years ago.
* The Hudson Bay store on one end is a fine department store that wears its age and history well.
* And when you consider all the other activities that Toronto has to offer within walking and transit distance, the Eaton Centre is just another interesting and fun choice within this vibrant metropolitan area.
* Most interesting is Trinity Square which is literally just outside the door near Sears and on the opposite side from Yonge.
$$
\textbf{Negative:}
$$
* Even at the Mall of America, the West Edmonton Mall, and this modernly designed and spacious Eaton Centre, the shopping experience soon becomes similar to any other big mall.


### Future Work 

Future work for the RNN:
A possible avenue to improve the outcomes of the neural network sentiment analysis is to utilize a CNN  instead of the a RNN. A CNN can take in ngrams append to the end of each sentence rather then the whole review for training. This form of input is closer to the output of the summariztions and could prove to create better results

Future work for the summarization:
Future work for the summarization would include making Opinosis work. This would allow us to do abstractive sumamries which could provide much more informational valude then extractive method. We could also try to run sentiment analysis on the data before summarization. This would categorize the data into positive and negative. This could potential provide us better summaries. Another thing we could do is perform some feature extraction.